In [1]:
## We taken the `enhanced-rag-intro.ipynb` and enhanced it use Elastic Search

In [1]:
!pip install minsearch

In [46]:
import minsearch
import json

from openai import OpenAI
import os
from dotenv import load_dotenv

In [47]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
docs_raw

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index (
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [12]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [39]:
def build_prompt(query, search_results):
    prompt_template = """
    You'are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering
    If CONTEXT doesn't have the answer, output NONE 
    
    QUESTION: 
    {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(
        question=query,
        context=context
    ).strip()

    return prompt

In [51]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[ { "role": "user", "content": prompt } ]
    )

    return response.choices[0].message.content

In [54]:
query = "how do i run Kafka"
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [55]:
answer

'To run Kafka (Java producer/consumer/kstreams/etc) in the terminal, go to the project directory and run the command:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf running Python Kafka files (like producer.py) and you get "Module \'kafka\' not found", create a virtual environment and install the requirements by running:\n\n```\npython -m venv env\nsource env/bin/activate       # (For Windows: env\\Scripts\\activate)\npip install -r ../requirements.txt\n```\n\nThen run the Python file inside that activated environment.'